# Environment Setup

In [48]:
# library imports
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from functools import reduce
import pickle

In [49]:
# sklearn imports
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin

In [50]:
# mounting google drive (running this code in colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
# setting up sql data path and engine
engine = create_engine("sqlite:////content/drive/MyDrive/Colab Notebooks/spdr.db")
conn = engine.connect()

In [52]:
# notebook settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Loading and Preprocessing the Data (as in Lecture)

In [53]:
# loading the data
query = text('SELECT * FROM ohlc_spdr')
ohlc = pd.read_sql_query(query, conn)
ohlc.shape
ohlc.head()

(67914, 8)

,ts,open,high,low,close,adj_close,volume,ticker
0,1993-01-29 00:00:00,43.96875,43.96875,43.75000,43.93750,24.840677,1003200.0,SPY
1,1993-02-01 00:00:00,43.96875,44.25000,43.96875,44.25000,25.017349,480500.0,SPY
2,1993-02-02 00:00:00,44.21875,44.37500,44.12500,44.34375,25.070364,201300.0,SPY
3,1993-02-03 00:00:00,44.40625,44.84375,44.37500,44.81250,25.335377,529400.0,SPY
4,1993-02-04 00:00:00,44.96875,45.09375,44.46875,45.00000,25.441385,531500.0,SPY


In [54]:
# functions from lecture
def vol_ohlc(df, lookback=10):
    o = df.open
    h = df.high
    l = df.low
    c = df.close

    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    close_vol = cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    open_vol = oc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    result = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    result.iloc[:lookback-1] = np.nan

    return result

In [55]:
tickers = ohlc.ticker.unique()

def df_merge(left, right):
    return pd.merge(left, right, on='ts', how='inner')

X = reduce(df_merge, [
    (lambda df:
    (
        df
        .assign(
            vol=vol_ohlc(df, lookback=5).fillna(0),
            cret=df.adj_close.pct_change()
        )[['ts', 'vol', 'cret']]
        .rename(columns={
            col: f'{col}_{ticker}' for col in ['ts', 'vol', 'cret'] if col != 'ts'
        })
    ))(ohlc[ohlc.ticker == ticker])
    for ticker in tickers
]).set_index('ts')

X.shape
X.tail()

(1393, 24)

,vol_SPY,cret_SPY,vol_XLC,cret_XLC,vol_XLY,cret_XLY,vol_XLP,cret_XLP,vol_XLE,cret_XLE,...,vol_XLI,cret_XLI,vol_XLB,cret_XLB,vol_XLRE,cret_XLRE,vol_XLK,cret_XLK,vol_XLU,cret_XLU
ts,,,,,,,,,,,,,,,,,,,,,
2023-12-22 00:00:00,0.128693,0.002010,0.171595,0.000138,0.189956,-0.007356,0.137244,0.007640,0.176196,0.002951,...,0.155127,0.004796,0.138895,0.006230,0.178772,0.004043,0.149241,0.001303,0.173567,0.003365
2023-12-26 00:00:00,0.123559,0.004223,0.161187,0.003856,0.187830,0.004235,0.129835,0.004212,0.171888,0.008590,...,0.158492,0.007072,0.139452,0.004673,0.170591,0.007299,0.142412,0.004166,0.158750,0.006068
2023-12-27 00:00:00,0.122910,0.001808,0.158523,0.000137,0.184354,0.003440,0.129718,0.004614,0.171189,-0.004434,...,0.156464,0.001492,0.127393,0.002442,0.172286,0.005747,0.146054,-0.000052,0.155162,-0.000635
2023-12-28 00:00:00,0.096539,0.000378,0.121723,0.002743,0.143163,-0.005197,0.099362,0.000418,0.166165,-0.015235,...,0.107992,0.000263,0.111944,-0.003828,0.148465,0.006211,0.130329,0.001193,0.138252,0.007147
2023-12-29 00:00:00,0.072901,-0.002895,0.097511,-0.006019,0.107239,-0.006169,0.072315,0.002087,0.160131,-0.002261,...,0.086568,-0.001227,0.083507,-0.003959,0.133526,-0.010864,0.093909,-0.003056,0.125436,-0.001262


In [56]:
y = X.vol_XLK.shift(-5)[:-5]
X = X[:-5]

In [57]:
X.shape
y.shape

(1388, 24)

(1388,)

In [58]:
pd.isnull(X).sum()

vol_SPY      0
cret_SPY     0
vol_XLC      0
cret_XLC     1
vol_XLY      0
cret_XLY     0
vol_XLP      0
cret_XLP     0
vol_XLE      0
cret_XLE     0
vol_XLF      0
cret_XLF     0
vol_XLV      0
cret_XLV     0
vol_XLI      0
cret_XLI     0
vol_XLB      0
cret_XLB     0
vol_XLRE     0
cret_XLRE    0
vol_XLK      0
cret_XLK     0
vol_XLU      0
cret_XLU     0
dtype: int64

In [ ]:
{col: y.corr(X[col]) for col in X.columns if X[col].dtype != 'object'}

{'vol_SPY': 0.7134699099998557,
 'cret_SPY': -0.17392498017137403,
 'vol_XLC': 0.6595170916771002,
 'cret_XLC': -0.14284324170926843,
 'vol_XLY': 0.6799315779640431,
 'cret_XLY': -0.17623413706684593,
 'vol_XLP': 0.7071712950367176,
 'cret_XLP': -0.11608784380824866,
 'vol_XLE': 0.6566647682081364,
 'cret_XLE': -0.15792268835883777,
 'vol_XLF': 0.6517292410615803,
 'cret_XLF': -0.16389695593637252,
 'vol_XLV': 0.7035810066492557,
 'cret_XLV': -0.12476511126055881,
 'vol_XLI': 0.6607142193385581,
 'cret_XLI': -0.180017276270817,
 'vol_XLB': 0.6346379611316387,
 'cret_XLB': -0.15073144850299885,
 'vol_XLRE': 0.6191616327638606,
 'cret_XLRE': -0.1352792944309433,
 'vol_XLK': 0.7152699653299577,
 'cret_XLK': -0.13240192000020784,
 'vol_XLU': 0.6246862091809325,
 'cret_XLU': -0.12385805523165071}

# Running Best Model from Lecture

In [59]:
# utilities from lecture
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

def evaluate_model(model, X, y, test_size=0.2):
    cv = TimeSeriesSplit(n_splits=int(y.shape[0] * test_size), test_size=3)
    scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

    return np.mean(cross_validate(model, X, y, cv=cv, scoring=scorer, n_jobs=-1, verbose = True)['test_score'])

In [ ]:
# loading the model
lecture_model_path = "/content/drive/MyDrive/Colab Notebooks/best_model.pkl"
lecture_model = pickle.load(open(lecture_model_path, "rb"))
lecture_model

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info plea

Pipeline(steps=[('feature_selector',
                 FeatureSelector(columns=Index(['vol_SPY', 'cret_SPY', 'vol_XLC', 'cret_XLC', 'vol_XLY', 'cret_XLY',
       'vol_XLP', 'cret_XLP', 'vol_XLE', 'cret_XLE', 'vol_XLF', 'cret_XLF',
       'vol_XLV', 'cret_XLV', 'vol_XLI', 'cret_XLI', 'vol_XLB', 'cret_XLB',
       'vol_XLRE', 'cret_XLRE', 'vol_XLK', 'cret_XLK', 'vol_XLU', 'cret_XLU'],
      dtype='object'))),
                ('impute', SimpleImputer(fill_value=0.0, strategy='constant')),
                ('scale', StandardScaler()), ('pca', PCA()),
                ('model', RandomForestRegressor())])

In [ ]:
evaluate_model(lecture_model, X, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 277 out of 277 | elapsed:  8.6min finished


-0.06697640631336647

# My Model

I have used the inelegant yet oft-used idea of throwing a more complex model at the problem. Specifically, I've used the LSTM model (followed by fully-connected layers) for predicting the volatility without any additional feature engineering. The result obtained **(-0.0634)** are marginally better than the baseline performance of the best model in the lecture (-0.0659 in the lecture or -0.067 as per the run above). 

It can certainly be improved with some feature engineering and/or hyperparameter tuning for a real-world usecase, but it outperforms the model in the lecture nonetheless for the purpose of the present assignment.

In [61]:
# defining an LSTM based estimator
class LSTMEstimator(BaseEstimator, RegressorMixin):
  def __init__(self):
    self.model = None

  def fit(self, X, y):
    self.model = keras.Sequential([
        keras.layers.LSTM(64, activation = "relu", input_shape = (X.shape[1], 1)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(32, activation = "relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10),
        keras.layers.Dense(1)
    ])
    self.model.compile(loss = "mse", optimizer = keras.optimizers.Adam())

    self.model.fit(
        X.reshape((X.shape[0], X.shape[1], 1)),
        y,
        epochs = 10,
        batch_size = 64,
        verbose = False #verbose = True
    )
    return self

  def predict(self, X):
    return self.model.predict(X.reshape(X.shape[0], X.shape[1], 1))

In [62]:
# creating an sklearn pipeline
my_model = Pipeline([
    ("impute", SimpleImputer(
        missing_values = np.nan,
        strategy = "constant",
        fill_value = 0.
    )),
    ("scale", StandardScaler()),
    ("estimator", LSTMEstimator())
])

In [69]:
# evaluating our model
evaluate_model(my_model, X, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done 277 out of 277 | elapsed: 26.9min finished
-0.06344041077084153
    
